- Load the nibabel file
- HU value in between -700 to 300. See if needed? or Just scale.
- Standardize in between 0-1
- Resample 256x256x200
- Create image/.npy dataset for the same
- Use the previous UNET arch to train the model

In [1]:
import nibabel as nib
import numpy as np
import os
from pathlib import Path
import torch
import nibabel.processing
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
root_path = Path("/scratch/scratch6/akansh12/Parse_data/train/train/")
train_image_file = list(root_path.glob("*/image/*"))
train_label_file = list(root_path.glob("*/label/*"))

In [3]:
def change_img_to_label_path(path):
    parts = list(path.parts)
    parts[parts.index("image")] = "label"  
    return Path(*parts)

def standardize(normalized):
    standardized = (normalized - normalized.min()) / (normalized.max() - normalized.min())
    return standardized 

def windowing(scan, window):
    min_value = window[0]
    max_value = window[1]
    return np.clip(scan,min_value, max_value)

def resample_ct(scan,shape, voxel_size, orientation):
    return nibabel.processing.conform(scan, shape, voxel_size, orientation = orientation)   

In [6]:
save_root = Path("/scratch/scratch6/akansh12/Parse_data/processed_train")

In [7]:
save_root = Path("/scratch/scratch6/akansh12/Parse_data/processed_train")
for counter, ct_file_path in enumerate(tqdm(train_image_file)):
    path_to_label = change_img_to_label_path(ct_file_path)
    
    ct_orig = nib.load(ct_file_path)
    ct_label = nib.load(path_to_label)
    
    #windowing
    ct_windowed = windowing(ct_orig.get_fdata(), [-700, 300])
    
    #Scaling
    standardized = standardize(ct_windowed)
       
    # Resample
    ct_orig = nib.Nifti1Image(standardized, ct_orig.affine)
    ct_orig_resampled = resample_ct(ct_orig, (256,256,200), (2,2,2), 'LPS')
    ct_label_resample = resample_ct(ct_label, (256,256,200), (2,2,2), 'LPS')
    
    if counter < 90:
        current_path = save_root/"train"/ct_file_path.parts[-3]
    else:
        current_path = save_root/"val"/ct_file_path.parts[-3]
    
    ct_orig_resampled_data = ct_orig_resampled.get_fdata()
    ct_label_resample_data = ct_label_resample.get_fdata()
    
    
    for i in tqdm(range(ct_orig_resampled_data.shape[-1])):
        ct_save_path = current_path/"image"
        ct_label_save_path = current_path/"label"
        ct_save_path.mkdir(parents=True, exist_ok=True)
        ct_label_save_path.mkdir(parents=True, exist_ok=True)

        np.save(ct_save_path/str(i), ct_orig_resampled_data[:,:,i])
        np.save(ct_label_save_path/str(i), ct_label_resample_data[:,:,i])

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]